# 3. (20 points)
We are given a dataset containing various criteria important to get admissions into Master’s program and probablity of getting an admit.
Dataset is available at http://preon.iiit.ac.in/~sanjoy_chowdhury/AdmissionDataset.zip

1. Implement a model using linear regression to predict the probablity of getting the admit.(10 points)
2. Compare the performance of **Mean square error loss function** vs **Mean Absolute error function** vs **Mean absolute percentage error function** and explain the reasons for the observed behaviour.(5 points)
3. Analyse and report the behaviour of the coefficients(for example: sign of coefficients, value of coefficients etc.) and support it with appropriate plots as necessary(5 points)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from utils.datatools import inspect_data


In [2]:
df = pd.read_csv("../data/admission/data.csv")

# we do not need Serial No. column
df.drop(['Serial No.'], axis=1, inplace=True)

# # make class label the last column 
# cols = df.columns.tolist()
# cols.remove("class")
# df = df[cols +["class"]]

# target = "label"
# # rename the column whose value is to be predicted to "label"
# df = df.rename(columns={"class": target})

df = df.rename(columns={"Chance of Admit ": "Chance of Admit"}) 

df.head(3)


,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,317,103,2,2.5,2.0,8.15,0,0.65
1,319,108,3,3.0,3.5,8.54,1,0.71
2,322,110,3,3.5,2.5,8.67,1,0.80


In [3]:
df.columns

Index(['GRE Score', 'TOEFL Score', 'University Rating', 'SOP', 'LOR ', 'CGPA',
       'Research', 'Chance of Admit'],
      dtype='object')

In [4]:
# inspect data to get insights
inspect_data(df)
df.info()

GRE Score 49 min: 290 max: 340
------------------
TOEFL Score 29 min: 92 max: 120
------------------
University Rating 5 min: 1 max: 5
[1 2 3 4 5]
[ 28 116 144  95  67]
------------------
SOP 9 min: 1.0 max: 5.0
[1.  1.5 2.  2.5 3.  3.5 4.  4.5 5. ]
[ 5 25 40 54 74 79 80 57 36]
------------------
LOR  9 min: 1.0 max: 5.0
[1.  1.5 2.  2.5 3.  3.5 4.  4.5 5. ]
[ 1  9 43 47 86 83 83 55 43]
------------------
CGPA 177 min: 7.2 max: 9.92
------------------
Research 2 min: 0 max: 1
[0 1]
[201 249]
------------------
Chance of Admit 60 min: 0.34 max: 0.97
------------------
Datframe size: 450
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 8 columns):
GRE Score            450 non-null int64
TOEFL Score          450 non-null int64
University Rating    450 non-null int64
SOP                  450 non-null float64
LOR                  450 non-null float64
CGPA                 450 non-null float64
Research             450 non-null int64
Chance of Admit  

In [5]:
# add a column for x0 containing all Ones
df = df.assign(x0=pd.Series(np.ones(len(df))).values)

# TODO: make x0 the first column
columns = df.columns.tolist()
columns.remove('x0')
df = df[['x0'] + columns]

In [6]:
df.head(3)

,x0,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1.0,317,103,2,2.5,2.0,8.15,0,0.65
1,1.0,319,108,3,3.0,3.5,8.54,1,0.71
2,1.0,322,110,3,3.5,2.5,8.67,1,0.80


In [29]:
import random

def train_test_split(X, y, test_size, random_state=None):
    """Splits data into training and testing sets.
    test_size must be fractional eg 0.2 for 20% split"""

    if random_state is not None:
        # Seed to generate same pseudo-random numbers everytime to make it reproducible.
        random.seed(random_state)

    test_size = round(test_size * len(X))  # change proportion to actual number of rows

    indices = X.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)

    X_test = X.loc[test_indices, :]
    y_test = y.loc[test_indices] 
    X_train = X.drop(test_indices)
    y_train = y.drop(test_indices)

    return X_train, X_test, y_train, y_test


In [30]:
X = df.drop(["Chance of Admit"], axis=1)
y = df["Chance of Admit"]


In [36]:
X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.2, random_state=1)

In [37]:
n = len(X_train.columns) # number of columns
# convert X and y from dataframe to numpy ndarrays
X_train = X_train.values
y_train = y_train.values
# make y a vector(1d column) 
y_train = np.reshape(y_train, (len(y_train), 1))

In [45]:
theta = np.zeros((n,1))
iterations = 3000
alpha = 0.000000001

In [46]:
def compute_cost(X, y, theta):
    """compute_cost(X, y, theta) computes the cost of using theta as the
    parameter for linear regression to fit the data points in X and y"""

    m = len(y) # number of training examples
    
    H = np.matmul(X, theta)  # H = hypothesis
    J = (1 / (2*m)) * (sum((H - y) ** 2))  # J = Cost function
    
    return J

In [47]:
def gradient_descent(X, y, theta, alpha, num_iters):
    """Performs gradient descent to learn theta
       theta = gradient_descent(x, y, theta, alpha, num_iters) updates theta by
       taking num_iters gradient steps with learning rate alpha
    """
    m = len(y) # number of training examples
    costs = np.zeros((num_iters, 1))

    for it in range(1, num_iters):
        H = np.matmul(X, theta)
        delta = (1/m) * np.matmul(X.T , (H - y))
        
        theta = theta - (alpha * delta)
        
        costs[it] = compute_cost(X, y, theta)
        
    return theta, costs

In [48]:
theta, costs = gradient_descent(X_train, y_train, theta, alpha, iterations)